# To run only on local not ssh

In [1]:
import scanpy as sc
import napari
import numpy as np
from matplotlib import cm
from matplotlib.colors import to_hex

# Load data
adata = sc.read_h5ad("/Users/mounim/Downloads/bins_IMMUNEX001.h5ad") 
coords = adata.obsm["spatial"]
coords = coords[:, [1, 0]]  # switch x and y

# Image and scalefactor
sample_id = list(adata.uns["spatial"].keys())[0]
he_img = adata.uns["spatial"][sample_id]["images"]["hires"]
scale = adata.uns["spatial"][sample_id]["scalefactors"]["tissue_hires_scalef"]

# Convert to pixel coordinates for the image
coords_pixels = coords * scale


# Convert image to uint8 if needed
if he_img.max() <= 1.0:
    he_img = (he_img * 255).astype(np.uint8)


In [2]:
# import imread
from skimage.io import imread
# -----------------------------
# 2. Load full-res image and compute scale factors
# -----------------------------
he_fullres = imread("/Users/mounim/Documents/IMMUNEX_data/IMAGE/HE_nanozoomer_tif/IMMUNEX001_Visium_HE_x40_z0.tif")
print("Image shape:", he_fullres.shape)

scale_factor_y = he_fullres.shape[0] / he_img.shape[0]
scale_factor_x = he_fullres.shape[1] / he_img.shape[1]
print("Scale factors:", scale_factor_y, scale_factor_x)

coords_hd = coords_pixels.copy()
coords_hd[:, 0] *= scale_factor_y  # y
coords_hd[:, 1] *= scale_factor_x  # x

Image shape: (91904, 119040, 3)
Scale factors: 19.841105354058723 19.84


In [3]:
# -----------------------------
# 3. Load and filter segmentation borders
# -----------------------------
from shapely.geometry import Polygon
import numpy as np
from skimage.measure import find_contours
from skimage.transform import AffineTransform
import napari

def is_valid_polygon(poly):
    if len(poly) < 3:
        return False
    try:
        p = Polygon(poly)
        return p.is_valid and p.area > 1e-3
    except:
        return False

# Load label image
labels = np.load("/Users/mounim/Downloads/he_segmentation_IMMUNEX001.npy")

# Extract contours and flip diagonally (swap x and y)
raw_shapes = [np.fliplr(c) for c in find_contours(labels, level=0.5)]
flipped_shapes = [s[:, [1, 0]] for s in raw_shapes]  # diagonal flip
filtered_shapes = [s for s in flipped_shapes if is_valid_polygon(s)]

# Build affine transform (based on coords_hd)
min_x, min_y = coords_hd.min(axis=0)
max_x, max_y = coords_hd.max(axis=0)

scale_x = (max_x - min_x) / labels.shape[1]
scale_y = (max_y - min_y) / labels.shape[0]
translation_x = min_x
translation_y = min_y

def compute_affine_scaling(source_shape, target_min, target_max):
    h, w = source_shape
    scale_x = (target_max[0] - target_min[0]) / (w - 1)
    scale_y = (target_max[1] - target_min[1]) / (h - 1)
    return scale_x, scale_y
target_min = coords_hd.min(axis=0)
target_max = coords_hd.max(axis=0)
scale_x, scale_y = compute_affine_scaling(labels.shape, target_min, target_max)
tform = AffineTransform(scale=(scale_x, scale_y), translation=target_min)

# Apply transform
aligned_shapes = [tform(s) for s in filtered_shapes]
# Optional: re-filter after transformation if needed
# aligned_shapes = [s for s in aligned_shapes if is_valid_polygon(s)]


/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/skimage/transform/_geometric.py:679: RuntimeWarning: divide by zero encountered in matmul
  dst = src @ matrix.T
/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/skimage/transform/_geometric.py:679: RuntimeWarning: overflow encountered in matmul
  dst = src @ matrix.T
/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/skimage/transform/_geometric.py:679: RuntimeWarning: invalid value encountered in matmul
  dst = src @ matrix.T


In [4]:
filtered_shapes

[array([[0.00000000e+00, 5.99999572e+01],
        [4.28082192e-05, 6.00000000e+01],
        [4.28082192e-05, 6.10000000e+01],
        [1.00000000e+00, 6.19999572e+01],
        [2.00000000e+00, 6.19999572e+01],
        [3.00000000e+00, 6.19999572e+01],
        [4.00000000e+00, 6.19999572e+01],
        [4.99995719e+00, 6.10000000e+01],
        [5.00000000e+00, 6.09999572e+01],
        [6.00000000e+00, 6.09999572e+01],
        [6.99995719e+00, 6.00000000e+01],
        [7.00000000e+00, 5.99999572e+01],
        [7.99995719e+00, 5.90000000e+01],
        [8.00000000e+00, 5.89999572e+01],
        [8.99995719e+00, 5.80000000e+01],
        [8.99995719e+00, 5.70000000e+01],
        [8.00000000e+00, 5.60000428e+01],
        [7.99995719e+00, 5.60000000e+01],
        [7.99995719e+00, 5.50000000e+01],
        [7.99995719e+00, 5.40000000e+01],
        [7.00000000e+00, 5.30000428e+01],
        [6.99995719e+00, 5.30000000e+01],
        [6.00000000e+00, 5.20000428e+01],
        [5.00000000e+00, 5.2000042

In [5]:
# -----------------------------
# 4. Launch Napari
# -----------------------------
viewer = napari.Viewer()
viewer.add_image(he_fullres, name="H&E Fullres", rgb=True)



/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/napari/_vispy/layers/scalar_field.py:195: UserWarning: data shape (91904, 119040, 3) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Image layer 'H&E Fullres' at 0x1788e4ca0>

In [6]:

# # Add Visium spot points
# features = adata.obs.loc[adata.obs.index[:len(coords_hd)], :].copy()
viewer.add_points(
    coords_hd,
    name="Spots",
    size=10,
    face_color='red',
    blending='translucent'
)

<Points layer 'Spots' at 0x3682dd8a0>

In [7]:
# Final filter on transformed polygons to ensure valid polygon shapes
from shapely.geometry import Polygon

def is_valid_polygon(poly):
    if len(poly) < 3:
        return False

    # Remove duplicates
    poly = np.unique(poly, axis=0)
    if len(poly) < 3:
        return False

    try:
        p = Polygon(poly)
        return p.is_valid and p.is_simple and p.area > 1e-3
    except Exception:
        return False


# Diagonal flip (transpose x and y)

# Add as POLYGONS
viewer.add_shapes(
    aligned_shapes,
    shape_type='polygon',  # 🟢 Polygon view (fillable)
    face_color='cyan',     # no fill
    edge_color='green',
    edge_width=1.0,
    name='aligned_shapes'
)

napari.run()


In [8]:
def get_bounds(polygons):
    all_points = np.vstack(polygons)
    min_x, min_y = all_points.min(axis=0)
    max_x, max_y = all_points.max(axis=0)
    return max_y - min_y, max_x - min_x  # height, width

seg_height, seg_width = get_bounds(aligned_shapes)

coords_min = coords_hd.min(axis=0)
coords_max = coords_hd.max(axis=0)
coords_height = coords_max[0] - coords_min[0]
coords_width = coords_max[1] - coords_min[1]

(height_width_comparison := {
    "Segmentation (H, W)": (seg_height, seg_width),
    "Coords_hd (H, W)": (coords_height, coords_width),
    "Difference (H, W)": (coords_height - seg_height, coords_width - seg_width)
})


{'Segmentation (H, W)': (np.float64(29552.679379128967),
  np.float64(29561.599666164264)),
 'Coords_hd (H, W)': (np.float64(29558.572054563883),
  np.float64(29555.706387157144)),
 'Difference (H, W)': (np.float64(5.892675434915873),
  np.float64(-5.893279007119418))}